In [1]:
import pandas as pd
import numpy as np
from utils import zebra
import re
solver_data = pd.read_excel("2020 Solver Partnership Matching, Campaign #1.xlsx", sheet_name='Solver_Data')
partner_data =  pd.read_excel("2020 Solver Partnership Matching, Campaign #1.xlsx", sheet_name='Partner_Data')
solver_data = solver_data.rename(columns={'Solution Name': 'Org'})
partner_data = partner_data.rename(columns={'Organization Name': 'Org',
                                            'Challenge':'Challenge Preference',
                                            'Stage': 'Solution Preference: Organization Stage',
                                            'Expertise': 'Partnership Preference: Non-Financial',
                                           'Geography': 'Geo Interests'})

partner_data = partner_data.replace(np.nan, "None")
solver_data  = solver_data.replace(np.nan, "None")


# Solver Data data Prep

### Get Geo regions 

In [2]:
solver_data['Geographies of Operation'] = solver_data['Geographies of Operation'].apply(lambda x : x.replace(';;',"") if ";;" in x else x )
solver_data['Geographies of Operation'] = solver_data['Geographies of Operation'].apply(lambda x: x.split(';'))

In [3]:
def extend_list(x, longest_list): 
    while len(x) < longest_list: 
        x.append(np.nan)
    return x
    
longest_list = solver_data['Geographies of Operation'].apply(len).max()
solver_data['Geographies of Operation'] = solver_data['Geographies of Operation'].apply(lambda x :  extend_list(x, longest_list) if len(x) < 3 else x )

In [4]:
geo_df = pd.DataFrame(solver_data['Geographies of Operation'].to_list(), columns=["Geo 1", "Geo 2", "Geo 3"])
geo_df.head()
solver_data[["Geo 1", "Geo 2", "Geo 3"]]= geo_df
solver_data = solver_data.drop(columns=['Geographies of Operation'])

### Modifying Tech column


In [5]:
solver_data_v2 = solver_data.copy()
solver_data_v2 = solver_data_v2.drop(columns=['Tech 2', 'Tech 3', 'Tech 4', 'Tech 5', 'Tech 6', 'Tech 7'])
solver_data_v2 = solver_data_v2.rename(columns={'Tech 1': 'Technology'})


In [6]:
solver_data_v2.columns

Index(['Challenge', 'Org', 'Key Need 1', 'Key Need 2', 'Key Need 3',
       'Key Need 4', 'Key Need 5', 'Key Need 6', 'Key Need 7', 'Key Need 8',
       'Type', 'Stage', 'HQ Region', 'Technology', 'Geo 1', 'Geo 2', 'Geo 3'],
      dtype='object')

In [7]:
solver_data_v2.head()

Challenge  \
0  2020 Health Security & Pandemics Challenge   
1  2020 Health Security & Pandemics Challenge   
2  2020 Health Security & Pandemics Challenge   
3  2020 Health Security & Pandemics Challenge   
4  2020 Health Security & Pandemics Challenge   

                                 Org  \
0                      AHSA Platform   
1     Biometricsfor vaccine delivery   
2       Jute-based biodegradable PPE   
3                          MapSights   
4  Mosquito-borne disease prevention   

                                          Key Need 1  \
0  Marketing, Media, and Exposure (e.g. branding/...   
1                                              Other   
2                        Legal or Regulatory Matters   
3                Attracting board members / advisors   
4  Funding and Financial (e.g. improving accounti...   

                                          Key Need 2  \
0                Attracting board members / advisors   
1                Attracting board members / advisors   
2  Business model (e.g. product-market fit, strat...   
3  Business model (e.g. product-market fit, strat...   
4                Attracting board members / advisors   

                                          Key Need 3  \
0  Business model (e.g. product-market fit, strat...   
1                                               None   
2                                               None   
3  Funding and Financial (e.g. improving accounti...   
4                                               None   

                    Key Need 4  \
0  Legal or Regulatory Matters   
1                         None   
2                         None   
3                         None   
4                         None   

                                          Key Need 5 Key Need 6 Key Need 7  \
0  Funding and Financial (e.g. improving accounti...       None       None   
1                                               None       None       None   
2                                               None       None       None   
3                                               None       None       None   
4                                               None       None       None   

  Key Need 8                                Type      Stage  \
0       None  Hybrid of for-profit and nonprofit      Pilot   
1       None                           Nonprofit      Pilot   
2       None                           Nonprofit  Prototype   
3       None                          For-Profit  Prototype   
4       None                          For-Profit     Growth   

                 HQ Region                                  Technology  \
0       Sub-Saharan Africa  Artificial Intelligence / Machine Learning   
1  Europe and Central Asia  Artificial Intelligence / Machine Learning   
2               South Asia              Biotechnology / Bioengineering   
3               South Asia  Artificial Intelligence / Machine Learning   
4            North America  Artificial Intelligence / Machine Learning   

                Geo 1       Geo 2 Geo 3  
0  Sub-Saharan Africa         NaN   NaN  
1  Sub-Saharan Africa  South Asia        
2          South Asia         NaN   NaN  
3          South Asia         NaN   NaN  
4       North America  South Asia

# Partner solver column Analysis
Going column by column to make sure that data is consistent

### Challenge column in Partner and Solver data

In [8]:
solver_data_v2['Challenge'] = solver_data_v2['Challenge'].apply(lambda x: x.replace("Challenge", "") if 'Challenge' in x else x)


Remove 2020 from challenge column solver data


In [9]:
solver_data_v2['Challenge'] = solver_data_v2['Challenge'].apply(lambda x: x.replace("2020", "") if '2020' in x else x)
solver_data_v2['Challenge'] = solver_data_v2['Challenge'].apply(lambda x: x.replace(' and ', ' & '))
solver_data_v2.head()

Challenge                                Org  \
0   Health Security & Pandemics                       AHSA Platform   
1   Health Security & Pandemics      Biometricsfor vaccine delivery   
2   Health Security & Pandemics        Jute-based biodegradable PPE   
3   Health Security & Pandemics                           MapSights   
4   Health Security & Pandemics   Mosquito-borne disease prevention   

                                          Key Need 1  \
0  Marketing, Media, and Exposure (e.g. branding/...   
1                                              Other   
2                        Legal or Regulatory Matters   
3                Attracting board members / advisors   
4  Funding and Financial (e.g. improving accounti...   

                                          Key Need 2  \
0                Attracting board members / advisors   
1                Attracting board members / advisors   
2  Business model (e.g. product-market fit, strat...   
3  Business model (e.g. product-market fit, strat...   
4                Attracting board members / advisors   

                                          Key Need 3  \
0  Business model (e.g. product-market fit, strat...   
1                                               None   
2                                               None   
3  Funding and Financial (e.g. improving accounti...   
4                                               None   

                    Key Need 4  \
0  Legal or Regulatory Matters   
1                         None   
2                         None   
3                         None   
4                         None   

                                          Key Need 5 Key Need 6 Key Need 7  \
0  Funding and Financial (e.g. improving accounti...       None       None   
1                                               None       None       None   
2                                               None       None       None   
3                                               None       None       None   
4                                               None       None       None   

  Key Need 8                                Type      Stage  \
0       None  Hybrid of for-profit and nonprofit      Pilot   
1       None                           Nonprofit      Pilot   
2       None                           Nonprofit  Prototype   
3       None                          For-Profit  Prototype   
4       None                          For-Profit     Growth   

                 HQ Region                                  Technology  \
0       Sub-Saharan Africa  Artificial Intelligence / Machine Learning   
1  Europe and Central Asia  Artificial Intelligence / Machine Learning   
2               South Asia              Biotechnology / Bioengineering   
3               South Asia  Artificial Intelligence / Machine Learning   
4            North America  Artificial Intelligence / Machine Learning   

                Geo 1       Geo 2 Geo 3  
0  Sub-Saharan Africa         NaN   NaN  
1  Sub-Saharan Africa  South Asia        
2          South Asia         NaN   NaN  
3          South Asia         NaN   NaN  
4       North America  South Asia

### Geo columns in Partner and solver data 

In [10]:
def split_collect(df_cols, delimiter=','):
    """
        Split each value in a cell based on multiple options 
        and return a list of unique options 
        
    """
    opts = df_cols.apply(lambda x : x.split(delimiter)).to_list()
    flatten_opts = [x for y in opts for x in y ]
    opts = pd.DataFrame(data=flatten_opts, columns=['options'])
    opts = opts['options'].value_counts().index.to_list()
    return opts

In [11]:
# Going by partner columns
geo_options = partner_data["Geo Interests"].apply(lambda x : x.split(",")).to_list()
flatten_geo_options = [x for y in geo_options for x in y ]
partner_geo_opts = pd.DataFrame(data=flatten_geo_options, columns=['geo_options'])
partner_geo_opts = partner_geo_opts['geo_options'].value_counts().index.to_list()

split_collect(partner_data['Geo Interests']), partner_geo_opts

(['North America',
  'Middle East and North Africa',
  'Europe and Central Asia',
  'Latin America and the Caribbean',
  'Sub-Saharan Africa',
  'East Asia and Pacific',
  'South Asia'],
 ['North America',
  'Middle East and North Africa',
  'Europe and Central Asia',
  'Latin America and the Caribbean',
  'Sub-Saharan Africa',
  'East Asia and Pacific',
  'South Asia'])

In [12]:
solver_geo_cols = solver_data_v2[['Geo 1', 'Geo 2', 'Geo 3']]
geo1_opts = solver_geo_cols['Geo 1'].value_counts().index.to_list()
geo2_opts = solver_geo_cols['Geo 2'].value_counts().index.to_list()
geo3_opts = solver_geo_cols['Geo 3'].value_counts().index.to_list()
solver_geo_opts = geo1_opts + geo2_opts + geo3_opts
solver_geo_opts = pd.DataFrame(data=solver_geo_opts, columns=['geo_options'])
solver_geo_opts = solver_geo_opts['geo_options'].value_counts().index.to_list()


In [13]:
partner_geo_opts

['North America',
 'Middle East and North Africa',
 'Europe and Central Asia',
 'Latin America and the Caribbean',
 'Sub-Saharan Africa',
 'East Asia and Pacific',
 'South Asia']

In [14]:
partner_geo_opts

['North America',
 'Middle East and North Africa',
 'Europe and Central Asia',
 'Latin America and the Caribbean',
 'Sub-Saharan Africa',
 'East Asia and Pacific',
 'South Asia']

**Geo options are OKAY!*** no issues here

### Stage columns in Partner and solver data 

In [15]:
partner_stage = partner_data['Solution Preference: Organization Stage']
partner_stage_opts = split_collect(partner_stage)


In [16]:
solver_stage_opts = solver_data_v2.Stage.value_counts().index.to_list()


In [17]:
set(partner_stage_opts) - set(solver_stage_opts)

{'Concept'}

In [18]:
solver_stage_opts, partner_stage_opts

(['Growth', 'Pilot', 'Prototype', 'Scale'],
 ['Growth', 'Scale', 'Pilot', 'Prototype', 'Concept'])

**Concept option not in solver stage** This is not an error but something to keep in mind


### Partnership columns in Partner and solver data 

In [19]:
# Partner data 
example_str = partner_data['Partnership Preference: Non-Financial'][1]
partner_data['Partnership Preference: Non-Financial'] = partner_data['Partnership Preference: Non-Financial'].replace(np.nan, "None", regex=True)
partnership_peferences = partner_data['Partnership Preference: Non-Financial'].to_list()
split_options = []
for single_str in partnership_peferences: 
    if single_str:        
        split_options.append(re.split(r',\s*(?![^()]*\))', single_str))

# split_options
flatten_popts = [x for y in split_options for x in y ]
popts = pd.DataFrame(data=flatten_popts, columns=['options'])
popts = popts['options'].value_counts().index.to_list()


In [20]:
popts

['Business model (e.g. product-market fit, strategy & development)',
 'Product / Service Distribution (e.g. expanding client base)',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'Human Capital (e.g. sourcing talent, board development, etc.)',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Other (explain below)',
 'Legal or Regulatory Matters',
 'None']

In [21]:
# Solver data 
all_needs_list = []
columns = ['Key Need 1',
           'Key Need 2',
           'Key Need 3',
           'Key Need 4',
           'Key Need 5',
           'Key Need 6',
           'Key Need 7',
           'Key Need 8' ] 
for x in columns: 
    all_needs_list.append(solver_data_v2[x].to_list())  

flatten_need_options = [x for y in all_needs_list for x in y ]
solver_need_opts = pd.DataFrame(data=flatten_need_options, columns=['options'])
list_need_options = solver_need_opts['options'].value_counts().index.to_list()
unique_need_options = []
for x in list_need_options:
    unique_need_options.append(x[0])
    


In [22]:
solver_need_options = set(unique_need_options)
partner_need_options = set(popts)

partner_need_options.difference(solver_need_options)

{'Business model (e.g. product-market fit, strategy & development)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Human Capital (e.g. sourcing talent, board development, etc.)',
 'Legal or Regulatory Matters',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'None',
 'Other (explain below)',
 'Product / Service Distribution (e.g. expanding client base)',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)'}

In [23]:
solver_need_options.difference(partner_need_options)

{'A', 'B', 'F', 'H', 'L', 'M', 'N', 'O', 'P', 'S', 'T'}

Looking at the difference between Solver data and partner data we find that there are options that are missing. Those options cannot really be dealt with. It seems like the option other is not appropriate dealt with and same with Human capital and Financial as well. 


In [24]:
# All PARTNER OPTIONS
partner_need_options

{'Business model (e.g. product-market fit, strategy & development)',
 'Financial (e.g. improving accounting practices, pitching to investors)',
 'Human Capital (e.g. sourcing talent, board development, etc.)',
 'Legal or Regulatory Matters',
 'Monitoring & Evaluation (e.g. collecting/using data, measuring impact)',
 'None',
 'Other (explain below)',
 'Product / Service Distribution (e.g. expanding client base)',
 'Public Relations (e.g. branding/marketing strategy, social and global media)',
 'Technology (e.g. software or hardware, web development/design, data analysis, etc.)'}

In [25]:
# ALL SOLVER OPTIONS
solver_need_options

{'A', 'B', 'F', 'H', 'L', 'M', 'N', 'O', 'P', 'S', 'T'}

In [26]:
need_errors = {'Other (explain below)':'Other', 
               'Human Capital (e.g. sourcing talent, board development, etc.)':'Human Capital (i.e. sourcing talent, board development, etc.)', 
               'Financial (e.g. improving accounting practices, pitching to investors)':'Funding and Financial (e.g. improving accounting practices, pitching to investors)'}

In [27]:
new_split_options = []
for nindx, row_opt in enumerate(split_options): 
    for indx, opt in enumerate(row_opt): 
        if opt in need_errors.keys(): 
            row_opt[indx] = need_errors[opt]
    new_split_options.append(row_opt)    

In [28]:
partners_need_df = pd.DataFrame(data=new_split_options, columns=['Key Need 1',
           'Key Need 2',
           'Key Need 3',
           'Key Need 4',
           'Key Need 5',
           'Key Need 6'])
partner_data_updated = pd.concat([partner_data, partners_need_df],axis=1)
partner_data_updated.head()

Organization ID                                Org First Name  \
0  0012M00002O876UQAR  American Student Assistance (ASA)      Alisa   
1  0014100001cYVK2AAO      BMW Foundation Herbert Quandt    Jessica   
2  0012M00002MvFHUQA3                       Clint Taylor      Clint   
3  0014100000hIt2nAAC                             Clorox       Lily   
4  0012M00002MvC5GQAV                           Conduent     Burhan   

    Last Name                                      Geo Interests  \
0       Wilke                                      North America   
1  von Farkas  Europe and Central Asia,Latin America and the ...   
2      Taylor      Latin America and the Caribbean,North America   
3        Dani  East Asia and Pacific,Latin America and the Ca...   
4      Jaffer  Europe and Central Asia,Middle East and North ...   

                                Challenge Preference  \
0  Good Jobs & Inclusive Entrepreneurship,Learnin...   
1  Good Jobs & Inclusive Entrepreneurship,Indigen...   
2  Good Jobs & Inclusive Entrepreneurship,Health ...   
3  Good Jobs & Inclusive Entrepreneurship,Health ...   
4  Health Security & Pandemics,Sustainable Food S...   

  Solution Preference: Organization Stage  \
0    Concept,Prototype,Pilot,Growth,Scale   
1                            Growth,Scale   
2    Concept,Prototype,Pilot,Growth,Scale   
3                      Pilot,Growth,Scale   
4            Prototype,Pilot,Growth,Scale   

                                                Tyle  \
0  For-Profit,Nonprofit,Hybrid of For-Profit & No...   
1        For-Profit,Hybrid of For-Profit & Nonprofit   
2                                         For-Profit   
3                   Hybrid of For-Profit & Nonprofit   
4  For-Profit,Nonprofit,Not Registered as Any Org...   

                                 Funding Preferences  \
0                       Grant funding,Equity funding   
1                              Other (explain below)   
2                                               None   
3  Grant funding,Equity funding,Other (explain be...   
4  Grant funding,Equity funding,Other (explain be...   

               Partnership Preference: Non-Financial  \
0  Business model (e.g. product-market fit, strat...   
1  Human Capital (e.g. sourcing talent, board dev...   
2  Business model (e.g. product-market fit, strat...   
3                                               None   
4  Business model (e.g. product-market fit, strat...   

                                Technology Expertise  \
0                                               None   
1                                               None   
2  Artificial Intelligence / Machine Learning,Big...   
3                                  Materials Science   
4  Artificial Intelligence / Machine Learning,Big...   

                                               Notes  \
0                                               None   
1  Leverage Knowledge in the Global BMW Foundatio...   
2                                               None   
3  Other partnering opportunities - Co-developmen...   
4                                               None   

                                          Key Need 1  \
0  Business model (e.g. product-market fit, strat...   
1  Human Capital (i.e. sourcing talent, board dev...   
2  Business model (e.g. product-market fit, strat...   
3                                               None   
4  Business model (e.g. product-market fit, strat...   

                                          Key Need 2  \
0  Monitoring & Evaluation (e.g. collecting/using...   
1                                              Other   
2  Monitoring & Evaluation (e.g. collecting/using...   
3                                               None   
4  Funding and Financial (e.g. improving accounti...   

                                          Key Need 3  \
0  Product / Service Distribution (e.g. expanding...   
1                                               None   
2  Technology (e.g. software or har

### Exporting to same excel file


In [29]:
file_name = '2020_data.xlsx'
with pd.ExcelWriter(file_name) as writer: 
    partner_data_updated.to_excel(writer, sheet_name='Partner Data', index=False)
    solver_data_v2.to_excel(writer, sheet_name='Solver Team Data', index= False)
writer.close()

# Testing weights generating function


In [30]:
def get_ch_solvers(solver_df):
    """ Get DataFrame of solver preferences 

    :param solver_df:  DataFrame contain the solver teams data
    :type solver_df: pandas.core.frame.DataFrame
    :return: An unpivoted list of solver preferences
    :rtype: pandas.core.frame.DataFrame
    """
    ch_solver = solver_df[["Org", "Challenge"]]
    ch_solver = ch_solver.apply(lambda x: x.str.strip() )
    ch_solver["Org"] = ch_solver["Org"].astype(str)
    
    return ch_solver

def get_ch_partners(partners_df): 
    """ Get challenge preferences for partners 

    :param partners_df: DataFrame containing partners preference data
    :type partners_df: pandas.core.frame.DataFrame
    :return: An unpivoted list of partner preferences
    :rtype: pandas.core.frame.DataFrame
    """
    ch_partners = partners_df[["Org","Challenge Preference"]]
    ch_partners["Challenge Preference"] =  ch_partners["Challenge Preference"].str.split(",")
    ch_partners_explode = ch_partners.explode("Challenge Preference")
    ch_partners_explode = ch_partners_explode.rename(columns={"Challenge Preference": "Challenge"})
    ch_partners_explode = ch_partners_explode.apply(lambda x: x.str.strip())
    ch_partners_explode.Challenge.astype(str)
    ch_partners_reset = ch_partners_explode.reset_index().drop('index', axis=1)
    ch_partners_reset["Org"] = ch_partners_reset["Org"].astype(str)
    
    return ch_partners_reset 

In [31]:
partners_df  = partner_data.copy()
solver_df = solver_data_v2.copy()
# Get a cleaned verison the challenges table from partners
ch_partners_challenges = get_ch_partners(partners_df)

# Get Solver Challenges
ch_solver = get_ch_solvers(solver_df)

# Merge partners and solvers challenge needs
merged_df = pd.merge(ch_solver,
                         ch_partners_challenges,
                         left_on="Challenge",
                         right_on='Challenge',
                         how='outer')
# Generate a pivot table on partners and solvers using the merged dataset                          
merged_pivot_table = pd.pivot_table(merged_df,
                                        values="Challenge",
                                        index=["Org_y"],
                                        columns=["Org_x"],
                                        aggfunc=np.sum)


# Set all the values on the pivot table to 1 and reset index  
challenges_pivot = merged_pivot_table.copy()
challenges_pivot[:] = 1
challenges_pivot_unpivoted = challenges_pivot.reset_index()

# Unpivot the pivot table acquire a list containing partner and solver matches
unpivoted_inital_table = pd.melt(challenges_pivot_unpivoted, id_vars="Org_y")
zero_column = unpivoted_inital_table['value']

# Assign new columns that contain inital scores for each key needs
unpivoted_inital_table = unpivoted_inital_table.assign(geo_weights=zero_column, 
                        challenge_weights=zero_column,
                        needs_weights=zero_column, 
                        stage_weights=zero_column)

# Drop the value column since it we only care about the 4 needs created above                        
partners_solvers_weights =  unpivoted_inital_table.drop(columns='value')


/home/pawan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
set4 = set(partners_solvers_weights['Org_y'].value_counts().index)
merged_df[merged_df['Org_y'] == 'Clorox']

Org_x  \
1                        AHSA Platform   
8       Biometricsfor vaccine delivery   
15        Jute-based biodegradable PPE   
22                           MapSights   
29   Mosquito-borne disease prevention   
36                         PENSA *660#   
43                                PODD   
68                                  D2   
81                   Humans inthe Loop   
94                             Nucleus   
107                           SOLshare   
120                  Someone Somewhere   
133                   TamoJunto.org.br   
146                      The Last Mile   

                                  Challenge   Org_y  
1               Health Security & Pandemics  Clorox  
8               Health Security & Pandemics  Clorox  
15              Health Security & Pandemics  Clorox  
22              Health Security & Pandemics  Clorox  
29              Health Security & Pandemics  Clorox  
36              Health Security & Pandemics  Clorox  
43              Health Security & Pandemics  Clorox  
68   Good Jobs & Inclusive Entrepreneurship  Clorox  
81   Good Jobs & Inclusive Entrepreneurship  Clorox  
94   Good Jobs & Inclusive Entrepreneurship  Clorox  
107  Good Jobs & Inclusive Entrepreneurship  Clorox  
120  Good Jobs & Inclusive Entrepreneurship  Clorox  
133  Good Jobs & Inclusive Entrepreneurship  Clorox  
146  Good Jobs & Inclusive Entrepreneurship  Clorox

In [33]:
solver_data_v2.head()

Challenge                                Org  \
0   Health Security & Pandemics                       AHSA Platform   
1   Health Security & Pandemics      Biometricsfor vaccine delivery   
2   Health Security & Pandemics        Jute-based biodegradable PPE   
3   Health Security & Pandemics                           MapSights   
4   Health Security & Pandemics   Mosquito-borne disease prevention   

                                          Key Need 1  \
0  Marketing, Media, and Exposure (e.g. branding/...   
1                                              Other   
2                        Legal or Regulatory Matters   
3                Attracting board members / advisors   
4  Funding and Financial (e.g. improving accounti...   

                                          Key Need 2  \
0                Attracting board members / advisors   
1                Attracting board members / advisors   
2  Business model (e.g. product-market fit, strat...   
3  Business model (e.g. product-market fit, strat...   
4                Attracting board members / advisors   

                                          Key Need 3  \
0  Business model (e.g. product-market fit, strat...   
1                                               None   
2                                               None   
3  Funding and Financial (e.g. improving accounti...   
4                                               None   

                    Key Need 4  \
0  Legal or Regulatory Matters   
1                         None   
2                         None   
3                         None   
4                         None   

                                          Key Need 5 Key Need 6 Key Need 7  \
0  Funding and Financial (e.g. improving accounti...       None       None   
1                                               None       None       None   
2                                               None       None       None   
3                                               None       None       None   
4                                               None       None       None   

  Key Need 8                                Type      Stage  \
0       None  Hybrid of for-profit and nonprofit      Pilot   
1       None                           Nonprofit      Pilot   
2       None                           Nonprofit  Prototype   
3       None                          For-Profit  Prototype   
4       None                          For-Profit     Growth   

                 HQ Region                                  Technology  \
0       Sub-Saharan Africa  Artificial Intelligence / Machine Learning   
1  Europe and Central Asia  Artificial Intelligence / Machine Learning   
2               South Asia              Biotechnology / Bioengineering   
3               South Asia  Artificial Intelligence / Machine Learning   
4            North America  Artificial Intelligence / Machine Learning   

                Geo 1       Geo 2 Geo 3  
0  Sub-Saharan Africa         NaN   NaN  
1  Sub-Saharan Africa  South Asia        
2          South Asia         NaN   NaN  
3          South Asia         NaN   NaN  
4       North America  South Asia

In [34]:
partner_data.head()

Organization ID                                Org First Name  \
0  0012M00002O876UQAR  American Student Assistance (ASA)      Alisa   
1  0014100001cYVK2AAO      BMW Foundation Herbert Quandt    Jessica   
2  0012M00002MvFHUQA3                       Clint Taylor      Clint   
3  0014100000hIt2nAAC                             Clorox       Lily   
4  0012M00002MvC5GQAV                           Conduent     Burhan   

    Last Name                                      Geo Interests  \
0       Wilke                                      North America   
1  von Farkas  Europe and Central Asia,Latin America and the ...   
2      Taylor      Latin America and the Caribbean,North America   
3        Dani  East Asia and Pacific,Latin America and the Ca...   
4      Jaffer  Europe and Central Asia,Middle East and North ...   

                                Challenge Preference  \
0  Good Jobs & Inclusive Entrepreneurship,Learnin...   
1  Good Jobs & Inclusive Entrepreneurship,Indigen...   
2  Good Jobs & Inclusive Entrepreneurship,Health ...   
3  Good Jobs & Inclusive Entrepreneurship,Health ...   
4  Health Security & Pandemics,Sustainable Food S...   

  Solution Preference: Organization Stage  \
0    Concept,Prototype,Pilot,Growth,Scale   
1                            Growth,Scale   
2    Concept,Prototype,Pilot,Growth,Scale   
3                      Pilot,Growth,Scale   
4            Prototype,Pilot,Growth,Scale   

                                                Tyle  \
0  For-Profit,Nonprofit,Hybrid of For-Profit & No...   
1        For-Profit,Hybrid of For-Profit & Nonprofit   
2                                         For-Profit   
3                   Hybrid of For-Profit & Nonprofit   
4  For-Profit,Nonprofit,Not Registered as Any Org...   

                                 Funding Preferences  \
0                       Grant funding,Equity funding   
1                              Other (explain below)   
2                                               None   
3  Grant funding,Equity funding,Other (explain be...   
4  Grant funding,Equity funding,Other (explain be...   

               Partnership Preference: Non-Financial  \
0  Business model (e.g. product-market fit, strat...   
1  Human Capital (e.g. sourcing talent, board dev...   
2  Business model (e.g. product-market fit, strat...   
3                                               None   
4  Business model (e.g. product-market fit, strat...   

                                Technology Expertise  \
0                                               None   
1                                               None   
2  Artificial Intelligence / Machine Learning,Big...   
3                                  Materials Science   
4  Artificial Intelligence / Machine Learning,Big...   

                                               Notes  
0                                               None  
1  Leverage Knowledge in the Global BMW Foundatio...  
2                                               None  
3  Other partnering opportunities - Co-developmen...  
4                                               None

## Testing total score generating function


In [35]:
input_file = 'input_data/2020_input_data.xlsx'
partner_solver_weights = pd.read_excel(input_file, sheet_name= 'Partner Solver Weights')
geo_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'geo_weights']], columns='Org_x', index='Org_y' )
needs_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'needs_weights']], columns='Org_x', index='Org_y' )
challenge_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'challenge_weights']], columns='Org_x', index='Org_y' )
stage_weights_pivot = pd.pivot(partner_solver_weights[['Org_y', 'Org_x', 'stage_weights']], columns='Org_x', index='Org_y' )


In [36]:
export_path = 'input_data/'
# print("total score export_path: ", export_path)
 # Mentor data
partners_df = zebra.csv_to_df(export_path+"partner_data.csv")
 # filled nan with 0 
partners_df = partners_df.fillna(0)
 
# Solver data
solver_df = zebra.csv_to_df(export_path+"solver_team_data.csv")
# filled nan with 0 
solver_df = solver_df.fillna(0)

# Get geo choice for solver
solver_geo = zebra.solver_geo_df(solver_df)

# Get geo choices for partner
partners_geo = zebra.partner_geo_df(partners_df)

# Generate geo contingency table
unpivoted_solver_geo = zebra.solver_regions_listform(solver_geo, solver_df)

# # Generate pivot table
# _,geo_pivot_copy = zebra.pivot_table_geo(unpivoted_solver_geo, partners_geo, export_path, export=True)

merged_geo = pd.merge(unpivoted_solver_geo,
                      partners_geo,
                      left_on="match",
                      right_on = "Geo Interests",
                      how='outer')
geo_pivot = pd.pivot_table(merged_geo,
                           index='Org_y',
                           columns='Org_x',
                           values='geo_match',
                           aggfunc=np.sum).fillna(0)
geo_pivot_values =  pd.pivot_table(merged_geo,
                                   index='Org_y',
                                   columns='Org_x',
                                   values='geo',
                                   aggfunc=np.sum).fillna(0)


geo_pivot_copy = geo_pivot_values.copy()
for col in geo_pivot_copy.columns: 
    geo_pivot_copy[col] = geo_pivot_copy[col].astype(str)
    geo_pivot_copy[col] = geo_pivot_copy[col].apply(lambda x: 0 if x.strip()== '0' else len(x.split()))
print(export_path)                                      

# Get needs for Partners
partners_needs = zebra.get_partners_needs(partners_df)

# Get needs for Solveras
unpivoted_solver_needs= zebra.get_solver_needs(solver_df)

# Read in partner pivot table
needs_values, needs_pivot_copy = zebra.pivot_table_needs(unpivoted_solver_needs,partners_needs, export_path, export=True)

# Get Partner Challenges
ch_partners_challenges = zebra.get_ch_partners(partners_df)

# Get Solver Challenges
ch_solver = zebra.get_ch_solvers(solver_df)

# Generate pivot table
challenges_pivot, challenges_pivot_copy = zebra.pivot_table_challenges(ch_solver, ch_partners_challenges, export_path, export=True)

# Get Parter stage data
st_partners = zebra.get_st_partners(partners_df)

# # Get Solver statge data
# st_solver = zebra.get_st_solver(solver_df)

# # Generate pivot table
# _,stage_pivot_copy = zebra.pivot_table_stage(st_solver, st_partners, export_path, export=True)

print("geo frames", geo_weights_pivot.shape, geo_pivot_copy.astype(float).shape) 
# geo_term = pd.DataFrame(geo_weights_pivot.values*geo_pivot_copy.astype(float).values, columns=geo_weights_pivot.columns, index=geo_weights_pivot.index)['geo_weights']
# stage_term = pd.DataFrame(stage_weights_pivot.values*stage_pivot_copy.astype(float).values, columns=stage_weights_pivot.columns, index=stage_weights_pivot.index)
# geo_stage_term = 100*pd.DataFrame(geo_term.values*stage_term.values, columns=stage_term.columns, index=stage_term.index)['stage_weights']
# needs_term =  pd.DataFrame(needs_weights_pivot.values*needs_pivot_copy.astype(float).values, columns=needs_weights_pivot.columns, index=needs_weights_pivot.index)['needs_weights']
# challenge_term = 10*pd.DataFrame(challenge_weights_pivot.values*challenges_pivot_copy.astype(float).values, columns=challenge_weights_pivot.columns, index=challenge_weights_pivot.index)['challenge_weights']


input_data/
geo frames (18, 43) (18, 42)


/media/pawan/e7/Data_science/colaberry/MIT_Solve_dashboard/notebooks/utils/zebra.py:256: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ch_partners["Challenge Preference"] =  ch_partners["Challenge Preference"].str.split(",")
/media/pawan/e7/Data_science/colaberry/MIT_Solve_dashboard/notebooks/utils/zebra.py:331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  st_partners["Stage"] = st_partners["Solution Preference: Organization Stage"].str.strip()
/media/pawan/e7/Data_science/colaberry/MIT_Solve_dashboard/noteb

In [37]:
geo_weights_pivot_set = set(geo_weights_pivot.index.values)
set_solvers2 = set(geo_pivot_copy.columns)


In [38]:
geo_pivot_copy_set = set(geo_pivot_copy.index.values)  # rows are partners
geo_weights_set = set(geo_weights_pivot.index.values)  # rows are partners
len(geo_pivot_copy_set), len(geo_weights_set)


(18, 18)

In [39]:
geo_weights_columns = geo_weights_pivot.columns.to_list()
geo_cols = []
for x in geo_weights_columns:
    _, solver = x
    geo_cols.append(solver)
geo_weights_columns_set = set(geo_cols)

geo_pivot_copy_columns_set = set(geo_pivot_copy.columns.values)
len(geo_weights_columns_set), len(geo_pivot_copy_columns_set)


(43, 42)

In [40]:
geo_pivot_copy_columns_set.difference(geo_weights_columns_set)

set()

In [45]:
merged_geo[merged_geo['Org_x'] == 'Sicangu Online Marketplace']

geo                       Org_x match geo_match Org_y Geo Interests
147   2  Sicangu Online Marketplace     0       2,0   NaN           NaN
148   3  Sicangu Online Marketplace     0       3,0   NaN           NaN
529   1  Sicangu Online Marketplace  None    1,None   NaN           NaN

In [42]:
geo_weights_columns_set.difference(geo_pivot_copy_columns_set)

{'Sicangu Online Marketplace'}

In [43]:
# set_solvers.difference(set_solvers2)


In [44]:
# merged_df[merged_df['Org_x'] == 'The Last Mile']